In [1]:
import requests
from newspaper import Article

# -------- CONFIG --------
SERPER_API_KEY = "b6377b6d2d07fe6a6f334f03986986351846d4bb"  # 🔁 Replace with your actual key
OLLAMA_API_URL = "http://localhost:11434/api/generate"

In [60]:
import re 

SyntaxError: invalid syntax (3161516611.py, line 1)

In [68]:
pip install playwright


   ---------------------------------------- 0.0/34.9 MB ? eta -:--:--
   ---------------------------------------- 0.3/34.9 MB ? eta -:--:--
    --------------------------------------- 0.8/34.9 MB 2.2 MB/s eta 0:00:16
    --------------------------------------- 0.8/34.9 MB 2.2 MB/s eta 0:00:16
   - -------------------------------------- 1.0/34.9 MB 1.3 MB/s eta 0:00:26
   - -------------------------------------- 1.3/34.9 MB 1.2 MB/s eta 0:00:28
   - -------------------------------------- 1.6/34.9 MB 1.3 MB/s eta 0:00:26
   -- ------------------------------------- 2.1/34.9 MB 1.5 MB/s eta 0:00:23
   -- ------------------------------------- 2.4/34.9 MB 1.4 MB/s eta 0:00:23
   --- ------------------------------------ 2.6/34.9 MB 1.4 MB/s eta 0:00:23
   --- ------------------------------------ 3.1/34.9 MB 1.5 MB/s eta 0:00:21
   ---- ----------------------------------- 3.7/34.9 MB 1.6 MB/s eta 0:00:20
   ---- ----------------------------------- 3.9/34.9 MB 1.6 MB/s eta 0:00:19
   ----- ----

In [97]:
def extract_article_text(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
        return f"Source: {url}\n\n{article.text}"
    except Exception as e:
        return f"Source: {url}\n\n[Failed to extract: {e}]"

In [98]:
def ask_llama3_locally(prompt):
    try:
        headers = {
            "Content-Type": "application/json",
            "Accept": "application/json"
        }
        response = requests.post(
            OLLAMA_API_URL,
            json={"model": "llama3", "prompt": prompt, "stream": False},
            headers=headers,
            verify=False  # Disable SSL verification for ngrok URL
        )
        if response.status_code == 200:
                answer_text = response.json().get("response", "[No response found]")
                return answer_text  # ✅ Return only the clean text
        else:
            return f"[HTTP {response.status_code}] Error from Llama3"
    except Exception as e:
        return f"[❌ Error calling Llama3: {e}]"

In [13]:
def is_url_trusted_by_ai(url):
    prompt = f"""You are a trustworthiness evaluator.

Decide whether the following website is generally considered a **professional and trustworthy source of information** for serious research, financial news, or due diligence.

Do **not** include websites that:
- Are editable by the public (like Wikipedia, Reddit, Quora)
- Are known for unverified or user-generated content

Only reply with `yes` or `no`.

### Website:
{url}

### Is this source trusted?
"""

    try:
        headers = {
            "Content-Type": "application/json",
            "Accept": "application/json"
        }
        response = requests.post(
            OLLAMA_API_URL,
            json={"model": "llama3", "prompt": prompt, "stream": False},
            headers=headers,
            verify=False
        )
        if response.status_code == 200:
            verdict = response.json().get("response", "").strip().lower()
            print(f"🤖 AI verdict for {url}: {verdict}")
            return "yes" in verdict
        else:
            return False
    except Exception as e:
        print(f"[AI trust check failed for {url}]: {e}")
        return False


In [99]:
def web_search_serper(query, max_results=5):
    headers = {
        "X-API-KEY": SERPER_API_KEY
    }
    json_data = {
        "q": query
    }
    response = requests.post("https://google.serper.dev/search", headers=headers, json=json_data)

    if response.status_code != 200:
        print(f"❌ Serper API error: {response.status_code}")
        return []

    data = response.json()
    results = data.get("organic", [])
    urls = []
    for item in results:
        url = item["link"]
        print(f"🔍 Evaluating source: {url}")
        if is_url_trusted_by_ai(url):
            urls.append(url)
        if len(urls) >= max_results:
            break
    return urls


In [100]:
def classify_question_type(query):
    classification_prompt = f"""You are a classification assistant.

Decide if the following question requires a short, direct answer (like a number, name, or fact), or a detailed, multi-paragraph summary.

Respond with only one word: `short` or `detailed`.

### Question:
{query}

### Answer Style:"""

    # Send to LLaMA3
    try:
        headers = {
            "Content-Type": "application/json",
            "Accept": "application/json"
        }
        response = requests.post(
            OLLAMA_API_URL,
            json={"model": "llama3", "prompt": classification_prompt, "stream": False},
            headers=headers,
            verify=False
        )
        if response.status_code == 200:
            style = response.json().get("response", "").strip().lower()
            return "short" if "short" in style else "detailed"
        else:
            return "detailed"
    except:
        return "detailed"


In [101]:
def display_final_answer(answer, query, urls):
    print("tetete"+answer)
    print("\n" + "="*80)
    print(f"🧠 Answer to: {query}")
    print("="*80)
    print("\n🔗 Sources Used:")
    for url in urls:
        print(f" - {url}")
    print("="*80)


In [102]:
def web_rag_query(query):
    print(f"🔍 Searching with Serper for: {query}")
    urls = web_search_serper(query)

    print(f"📎 URLs found: {urls}")
    if not urls:
        return "[❌ No articles found for this query.]", urls

    print(f"📄 Extracting content from {len(urls)} URLs...")
    sources = [extract_article_text(url) for url in urls]

    answer_style = classify_question_type(query)

    # Build the final prompt
    if answer_style == "short":
        final_prompt = f"""You are a precise assistant. Give a short, factual answer based **only** on the provided sources.

### Question:
{query}

### Sources:
{''.join(sources)}

### Answer:
"""
    else:
        final_prompt = f"""You are an expert crypto analyst and news summarizer.

Please answer the following question using only the information from the provided sources.
Be clear, concise, and neutral in tone.

### Question:
{query}

### Sources:
{''.join(sources)}

### Summary:
"""

    print("🧠 Asking llama3 locally...")
    answer = ask_llama3_locally(final_prompt)
    #print(f"🤖 Llama3 answer: {answer.strip()}")
    return answer, urls


In [103]:
question = "give me the exact price of bitcoin today ?"
final_answer, urls = web_rag_query(question)
display_final_answer(final_answer, question, urls)


🔍 Searching with Serper for: give me the exact price of bitcoin today ?
🔍 Evaluating source: https://www.coindesk.com/price/bitcoin
🤖 AI verdict for https://www.coindesk.com/price/bitcoin: yes
🔍 Evaluating source: https://bitflyer.com/en-us/bitcoin-chart
🤖 AI verdict for https://bitflyer.com/en-us/bitcoin-chart: yes
🔍 Evaluating source: https://coinmarketcap.com/currencies/bitcoin/
🤖 AI verdict for https://coinmarketcap.com/currencies/bitcoin/: **no**
🔍 Evaluating source: https://www.binance.com/en/price/bitcoin
🤖 AI verdict for https://www.binance.com/en/price/bitcoin: no
🔍 Evaluating source: https://www.coinbase.com/price/bitcoin
🤖 AI verdict for https://www.coinbase.com/price/bitcoin: no
🔍 Evaluating source: https://ca.finance.yahoo.com/quote/BTC-USD/
🤖 AI verdict for https://ca.finance.yahoo.com/quote/BTC-USD/: yes
🔍 Evaluating source: https://www.coinbase.com/converter/btc/usd
🤖 AI verdict for https://www.coinbase.com/converter/btc/usd: no
🔍 Evaluating source: https://bitbo.io/
🤖 